## Kaggel: Digit Recognizer(MNIST)  by Hyperopt + Data Augmentaion 
Kaggle Digit recognizer: https://www.kaggle.com/c/digit-recognizer  
Hyperopt: https://github.com/hyperopt/hyperopt  

### Score: 0.99671
* max_evals= 20 (time: 2h 5m)  
* data_augmentation (time: 3m 17s) 

Python 3.6  
NVIDIA GTX1060  

In [51]:
import warnings
warnings.filterwarnings('ignore')

import hyperopt
from hyperopt import hp, fmin, rand, tpe, Trials, space_eval, STATUS_OK

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils import np_utils
import keras

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# fix random seed
import tensorflow as tf
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '0'
seed = 123
rn.seed(seed)
np.random.seed(seed)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
from keras import backend as K
tf.set_random_seed(seed)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

print('tensorflow ver.:',tf.__version__)
print('keras ver.     : ', keras.__version__)
print('hyperopt ver.  :   ', hyperopt.__version__)

tensorflow ver.: 1.11.0
keras ver.     :  2.2.2
hyperopt ver.  :    0.2


## Data preparation: MNIST from Kaggle

In [52]:
train = pd.read_csv('../train.csv')
label = train.label
train = train.drop(['label'], axis=1)
train = train.values.reshape(-1, 28, 28, 1)

x_train, x_val, y_train, y_val = train_test_split(train, label, test_size=0.15, shuffle=True, random_state=seed)
x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0

[x_train.shape, x_val.shape]

[(35700, 28, 28, 1), (6300, 28, 28, 1)]

## Hyperparameters for Hyperopt:  
* Mainly each Dropout ratio

In [53]:
params = {
    'Dropout_0':        hp.uniform('Dropout_0', 0.0, 0.5),
    'Dropout_1':        hp.uniform('Dropout_1', 0.0, 0.5),
    'Dropout_2':        hp.uniform('Dropout_2', 0.0, 0.5),
    'Dropout_3':        hp.uniform('Dropout_3', 0.0, 0.5),
    'Dropout_4':        hp.uniform('Dropout_4', 0.0, 0.5)
}

batch_size = 64

## CNN Model:

In [54]:
cnt = 0
def cnn_model(params):
    
    initializer = keras.initializers.glorot_uniform(seed=seed)
    
    model = Sequential() 
        
    model.add(Conv2D(32*2, (5,5), padding='same', activation='relu', kernel_initializer=initializer, input_shape=(28,28,1)))
    model.add(Conv2D(32*2, (5,5), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(params['Dropout_0'], seed=seed))
    
    model.add(Conv2D(64*2, (3,3), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(Conv2D(64*2, (3,3), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(params['Dropout_1'], seed=seed))
    
    model.add(Conv2D(128*2, (3,3), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(Conv2D(128*2, (3,3), padding='same', activation='relu', kernel_initializer=initializer))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(params['Dropout_2'], seed=seed))

    model.add(Flatten())
    model.add(Dense(512, activation="relu", kernel_initializer=initializer))
    model.add(BatchNormalization())
    model.add(Dropout(params['Dropout_3'], seed=seed))
    model.add(Dense(128, activation = "relu", kernel_initializer=initializer))
    model.add(BatchNormalization())
    model.add(Dropout(params['Dropout_4'], seed=seed))
    
    model.add(Dense(10, activation = "softmax", kernel_initializer=initializer))

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5,verbose=1, cooldown=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

    hist = model.fit(x_train, y_train,
                     batch_size=batch_size,
                     epochs=50,
                     verbose=1,
                     shuffle=True,
                     validation_data=(x_val, y_val),
                     callbacks=[reduce_lr, early_stopping])
    
    loss = hist.history['val_loss'][-1]
    acc = hist.history['val_acc'][-1]
    
    global cnt
    print(cnt, ': Val loss:', loss, ': Val acc:', acc, '\n\n')
    cnt += 1
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model, 'hist': hist}
    

## Search the Hyperparameters & the Best model:

In [55]:
trials = Trials()
best = fmin(fn=cnn_model, 
            space=params, 
            algo=tpe.suggest, 
            max_evals=20, # 50: 5h 15m
            trials=trials,
            verbose=1,
            rstate=np.random.RandomState(seed))

best

Train on 35700 samples, validate on 6300 samples
Epoch 1/50
35700/35700 [==============================] - 25s 692us/step - loss: 0.2446 - acc: 0.9233 - val_loss: 0.0789 - val_acc: 0.9762
Epoch 2/50
35700/35700 [==============================] - 17s 484us/step - loss: 0.0747 - acc: 0.9775 - val_loss: 0.0652 - val_acc: 0.9811
Epoch 3/50
35700/35700 [==============================] - 17s 486us/step - loss: 0.0584 - acc: 0.9826 - val_loss: 0.0393 - val_acc: 0.9879
Epoch 4/50
35700/35700 [==============================] - 17s 485us/step - loss: 0.0501 - acc: 0.9847 - val_loss: 0.0325 - val_acc: 0.9898
Epoch 5/50
35700/35700 [==============================] - 17s 485us/step - loss: 0.0414 - acc: 0.9878 - val_loss: 0.0369 - val_acc: 0.9895
Epoch 6/50
35700/35700 [==============================] - 17s 485us/step - loss: 0.0430 - acc: 0.9871 - val_loss: 0.0386 - val_acc: 0.9883

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/50
35700/35700 [============

35700/35700 [==============================] - 18s 500us/step - loss: 0.0125 - acc: 0.9965 - val_loss: 0.0165 - val_acc: 0.9956
Epoch 13/50
35700/35700 [==============================] - 18s 493us/step - loss: 0.0083 - acc: 0.9978 - val_loss: 0.0217 - val_acc: 0.9941
Epoch 14/50
35700/35700 [==============================] - 18s 492us/step - loss: 0.0076 - acc: 0.9978 - val_loss: 0.0188 - val_acc: 0.9951

Epoch 00014: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 15/50
35700/35700 [==============================] - 18s 494us/step - loss: 0.0060 - acc: 0.9985 - val_loss: 0.0197 - val_acc: 0.9952
Epoch 16/50
35700/35700 [==============================] - 18s 498us/step - loss: 0.0057 - acc: 0.9984 - val_loss: 0.0145 - val_acc: 0.9962
Epoch 17/50
35700/35700 [==============================] - 18s 495us/step - loss: 0.0038 - acc: 0.9987 - val_loss: 0.0170 - val_acc: 0.9959
Epoch 18/50
35700/35700 [==============================] - 18s 501us/step - loss: 0.0036 - 

Epoch 14/50
35700/35700 [==============================] - 18s 503us/step - loss: 0.0096 - acc: 0.9970 - val_loss: 0.0182 - val_acc: 0.9949
Epoch 15/50
35700/35700 [==============================] - 18s 502us/step - loss: 0.0077 - acc: 0.9979 - val_loss: 0.0251 - val_acc: 0.9937

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
Epoch 16/50
35700/35700 [==============================] - 18s 507us/step - loss: 0.0054 - acc: 0.9986 - val_loss: 0.0196 - val_acc: 0.9952
Epoch 17/50
35700/35700 [==============================] - 18s 501us/step - loss: 0.0046 - acc: 0.9986 - val_loss: 0.0180 - val_acc: 0.9952

Epoch 00017: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
Epoch 18/50
35700/35700 [==============================] - 18s 502us/step - loss: 0.0032 - acc: 0.9990 - val_loss: 0.0181 - val_acc: 0.9954
Epoch 00018: early stopping
7 : Val loss: 0.01814576182998569 : Val acc: 0.9953968253968254 


Train on 35700 samples, validate on 6300 sam

Epoch 10/50
35700/35700 [==============================] - 18s 510us/step - loss: 0.0196 - acc: 0.9939 - val_loss: 0.0271 - val_acc: 0.9917

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
Epoch 11/50
35700/35700 [==============================] - 18s 508us/step - loss: 0.0130 - acc: 0.9957 - val_loss: 0.0189 - val_acc: 0.9949
Epoch 12/50
35700/35700 [==============================] - 18s 508us/step - loss: 0.0102 - acc: 0.9968 - val_loss: 0.0193 - val_acc: 0.9944
Epoch 13/50
35700/35700 [==============================] - 18s 510us/step - loss: 0.0104 - acc: 0.9968 - val_loss: 0.0183 - val_acc: 0.9946
Epoch 14/50
35700/35700 [==============================] - 18s 513us/step - loss: 0.0093 - acc: 0.9972 - val_loss: 0.0184 - val_acc: 0.9949
Epoch 15/50
35700/35700 [==============================] - 18s 508us/step - loss: 0.0086 - acc: 0.9973 - val_loss: 0.0195 - val_acc: 0.9951

Epoch 00015: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814

35700/35700 [==============================] - 18s 517us/step - loss: 0.0051 - acc: 0.9985 - val_loss: 0.0163 - val_acc: 0.9959
Epoch 17/50
35700/35700 [==============================] - 18s 514us/step - loss: 0.0047 - acc: 0.9985 - val_loss: 0.0147 - val_acc: 0.9957

Epoch 00017: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
Epoch 18/50
35700/35700 [==============================] - 18s 515us/step - loss: 0.0038 - acc: 0.9989 - val_loss: 0.0137 - val_acc: 0.9962
Epoch 19/50
35700/35700 [==============================] - 18s 514us/step - loss: 0.0040 - acc: 0.9987 - val_loss: 0.0139 - val_acc: 0.9960
Epoch 20/50
35700/35700 [==============================] - 18s 514us/step - loss: 0.0031 - acc: 0.9990 - val_loss: 0.0145 - val_acc: 0.9959

Epoch 00020: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 21/50
35700/35700 [==============================] - 18s 513us/step - loss: 0.0027 - acc: 0.9993 - val_loss: 0.0147 - val_acc: 0.9960
Epoch 22/

Epoch 24/50
35700/35700 [==============================] - 18s 518us/step - loss: 0.0022 - acc: 0.9994 - val_loss: 0.0156 - val_acc: 0.9962

Epoch 00024: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 25/50
35700/35700 [==============================] - 19s 520us/step - loss: 0.0024 - acc: 0.9994 - val_loss: 0.0152 - val_acc: 0.9965
Epoch 00025: early stopping
14 : Val loss: 0.01518195620736122 : Val acc: 0.9965079365079365 


Train on 35700 samples, validate on 6300 samples
Epoch 1/50
35700/35700 [==============================] - 27s 763us/step - loss: 0.1509 - acc: 0.9543 - val_loss: 0.0606 - val_acc: 0.9806
Epoch 2/50
35700/35700 [==============================] - 19s 519us/step - loss: 0.0564 - acc: 0.9833 - val_loss: 0.0740 - val_acc: 0.9778
Epoch 3/50
35700/35700 [==============================] - 19s 519us/step - loss: 0.0424 - acc: 0.9871 - val_loss: 0.0342 - val_acc: 0.9895
Epoch 4/50
35700/35700 [==============================] - 19s 522us/step - l

Epoch 16/50
35700/35700 [==============================] - 19s 533us/step - loss: 0.0050 - acc: 0.9985 - val_loss: 0.0130 - val_acc: 0.9960
Epoch 17/50
35700/35700 [==============================] - 19s 533us/step - loss: 0.0038 - acc: 0.9990 - val_loss: 0.0151 - val_acc: 0.9960
Epoch 18/50
35700/35700 [==============================] - 19s 531us/step - loss: 0.0039 - acc: 0.9989 - val_loss: 0.0169 - val_acc: 0.9956

Epoch 00018: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
Epoch 19/50
35700/35700 [==============================] - 19s 533us/step - loss: 0.0032 - acc: 0.9991 - val_loss: 0.0154 - val_acc: 0.9956
Epoch 20/50
35700/35700 [==============================] - 19s 535us/step - loss: 0.0028 - acc: 0.9991 - val_loss: 0.0146 - val_acc: 0.9960

Epoch 00020: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
Epoch 21/50
35700/35700 [==============================] - 19s 533us/step - loss: 0.0025 - acc: 0.9993 - val_loss: 0.0145 - val_acc: 0.99

{'Dropout_0': 0.29949679088221454,
 'Dropout_1': 0.3191838155900335,
 'Dropout_2': 0.20843884641241,
 'Dropout_3': 0.17705224022098692,
 'Dropout_4': 0.28130745896863774}

## The Best Hyperparameters:

In [56]:
space_eval(params, best)

{'Dropout_0': 0.29949679088221454,
 'Dropout_1': 0.3191838155900335,
 'Dropout_2': 0.20843884641241,
 'Dropout_3': 0.17705224022098692,
 'Dropout_4': 0.28130745896863774}

## The Best Result

In [57]:
trials.best_trial['result']

{'hist': <keras.callbacks.History at 0x7f9983136f98>,
 'loss': -0.9966666666666667,
 'model': <keras.engine.sequential.Sequential at 0x7f9984ed6160>,
 'status': 'ok'}

## The Best model:

In [58]:
best_model = trials.best_trial['result']['model']
#best_model.save('hyperopt_best_model.hdf5')

## Evaluation before Data Augumantation

In [59]:
score = best_model.evaluate(x_val, y_val, verbose=1)
score

6300/6300 [==============================] - 2s 357us/step


[0.013626170573792593, 0.9966666666666667]

## Data Augmentation:

In [106]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.1)

datagen.fit(x_train, augment=True, seed=seed)

reduce_lr_aug = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)
early_stop_aug = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

hist_gen = best_model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), 
                                    epochs=50, 
                                    validation_data=(x_val, y_val),
                                    steps_per_epoch=x_train.shape[0]//batch_size,
                                    callbacks=[reduce_lr_aug, early_stop_aug],
                                    verbose=2)

loss_aug = hist_gen.history['val_loss'][-1]
acc_aug = hist_gen.history['val_acc'][-1]

print('Loss_aug: ', loss_aug, ', Acc_aug: ', acc_aug, '\n\n')

Epoch 1/50
 - 18s - loss: 0.0181 - acc: 0.9940 - val_loss: 0.0118 - val_acc: 0.9962
Epoch 2/50
 - 18s - loss: 0.0170 - acc: 0.9948 - val_loss: 0.0117 - val_acc: 0.9960
Epoch 3/50
 - 18s - loss: 0.0161 - acc: 0.9950 - val_loss: 0.0119 - val_acc: 0.9959

Epoch 00003: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.
Epoch 4/50
 - 18s - loss: 0.0168 - acc: 0.9944 - val_loss: 0.0118 - val_acc: 0.9963
Epoch 5/50
 - 18s - loss: 0.0156 - acc: 0.9950 - val_loss: 0.0118 - val_acc: 0.9963

Epoch 00005: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-06.
Epoch 6/50
 - 18s - loss: 0.0182 - acc: 0.9943 - val_loss: 0.0118 - val_acc: 0.9962
Epoch 7/50
 - 18s - loss: 0.0175 - acc: 0.9944 - val_loss: 0.0117 - val_acc: 0.9963

Epoch 00007: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.
Epoch 8/50
 - 18s - loss: 0.0163 - acc: 0.9949 - val_loss: 0.0116 - val_acc: 0.9963
Epoch 9/50
 - 18s - loss: 0.0176 - acc: 0.9948 - val_loss: 0.0119 - val_acc: 0.9960
E

## Evaluation after Data Augmentation:

In [107]:
score_aug = best_model.evaluate(x_val, y_val, verbose=1)
print('Before augmentation: ', score)
print('After  augmentation: ', score_aug)

6300/6300 [==============================] - 2s 354us/step
Before augmentation:  [0.013626170573792593, 0.9966666666666667]
After  augmentation:  [0.011852738058484632, 0.9963492063492063]


## Prediction for the Submission:

In [108]:
test = pd.read_csv('../test.csv')
test_index = test.index
test = test.values.reshape(-1, 28, 28, 1).astype('float32') / 255.0

pred = best_model.predict(test, verbose=1)
result = pred.argmax(axis=1)

28000/28000 [==============================] - 10s 342us/step


## Output the Submission csv file:

In [104]:
submission = pd.DataFrame({'ImageId': test_index+1, 'Label': result})
submission.to_csv('hyperopt_augment_submission.csv', index=False)

#### Accuracy estimation: comparison with the previous result scored at 0.99671

In [105]:
prev_cnn = pd.read_csv('hyperopt_augment_submission_99671.csv', index_col=0)
res = pd.read_csv('hyperopt_augment_submission.csv', index_col=0)
match_num = np.sum(prev_cnn.Label.values == res.Label.values)
acc = match_num / len(res) #* 0.99671
print('Approx. accuracy: {0:.5f}'.format(acc))

Approx. accuracy: 1.00000
